In [56]:
import pandas as pd
#from readability import Document


#for NER
import en_core_web_sm
nlp= en_core_web_sm.load()
import re
import sys
import nltk
import wikipediaapi
from nltk.tokenize import sent_tokenize, RegexpTokenizer, word_tokenize

import spacy
from spacy.matcher import Matcher

nltk.download('punkt', quiet=True)

True

In [10]:
def get_user_input():
    userInput = input("> ")
    return userInput

In [11]:
def end_condition(text):
    tmptxt = text
    pattern = r'[^A-Za-z]' # Remove non-letter characters (I nuke the punctuation)
    tmptxt = re.sub(pattern, '', tmptxt)
    tmptxt = tmptxt.lower()
    if tmptxt == 'exit':
        return False
    else:
        return True

In [12]:
#Leahs scraping code from here
def fetch_wikipedia_summary(topic):
    user_agent = 'PA2/1.0 (lantler@gmu.edu)'
    wiki_api = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})
    page = wiki_api.page(topic)
    if page.exists():
        return page.summary
    else:
        return ""

In [16]:
def correct_tags(key_words):
    corrected_tags = []
    pos_tokens = nltk.word_tokenize(key_words)
    tags = nltk.pos_tag(pos_tokens)

    # Rule-based disambiguation
    for i, (word, tag) in enumerate(tags):
        if word.lower() == "fall" and i > 0 and tags[i-1][1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            corrected_tags.append((word, 'VB'))
        elif word.lower() == "die" and i > 0 and tags[i-1][1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            corrected_tags.append((word, 'VB'))
        elif word.lower() == "born" and i > 0 and tags[i-1][1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            corrected_tags.append((word, 'VB'))
        elif word.lower() == "start" and i > 0 and tags[i-1][1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            corrected_tags.append((word, 'VB'))
        else:
            corrected_tags.append((word, tag))
    return corrected_tags

In [17]:
def find_key_words(query):
    unimportant_words = r"\b(([Ww]here|[Ww]hat|[Ww]ho|[Ww]hen) (is|was|did))( (a|the))?\b"
    key_words = re.sub(unimportant_words, "", query)
    return key_words

In [18]:
def find_noun(corrected_tags):
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'IN']

    # Filter out only the nouns
    nouns = [word for word, pos in corrected_tags if pos in noun_tags]
    noun = " ".join(nouns)

    return noun

In [19]:
def gen_search_terms(query):
    # Find the key words
    key_words = find_key_words(query)
    
    # Then, Create tags on these key words
    corrected_tags = correct_tags(key_words)

    # Then, find nouns
    noun = find_noun(corrected_tags)

    # Join nouns
    search_term = "".join(noun)
    
    return search_term

In [24]:
#Leahs scraping code from here
def fetch_wikipedia_summary(topic):
    user_agent = 'PA2/1.0 (lantler@gmu.edu)'
    wiki_api = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})
    page = wiki_api.page(topic)
    if page.exists():
        return page.summary
    else:
        return ""

In [65]:
def entity_search(text):
    # Perform NER on the text
    doc = nlp(text)
    # Get the named entities
    if doc.ents:
        # Assign the first named entity to the global variable entity
        entity = doc.ents[0].text
    else:
        # If no named entities are found, set entity to an empty string or other default value
        entity = ""
    return entity

def get_first_date(text):
    doc = nlp(text)

    first_date = None
    # Iterate through entities to find the first date
    for ent in doc.ents:
        if ent.label_ == 'DATE':
            first_date = ent.text
            break  # Stop as soon as the first date is found
    
    return first_date

def find_sentence_with_date(text):
    doc = nlp(text)
    
    matcher = Matcher(nlp.vocab)
    matcher.add("DATE", [[{"ENT_TYPE": "DATE"}]])
    
    for sentence in doc.sents:
        sentence_doc = nlp(sentence.text)
        matches = matcher(sentence_doc)
        if matches:
            return sentence.text # Match sentence if it has a date
    
    return None 

def find_sentence_with_regex(text, regex_pattern):
    doc = nlp(text)
    
    for sentence in doc.sents:
        if re.search(regex_pattern, sentence.text):
            return sentence.text
    
    return None 

In [67]:
def gen_who_response(user_input, wikipedia_summary):
    # Return the first sentence of the wikipedia summary
    doc = nlp(wikipedia_summary)
    # Return the first sentence
    for sent in doc.sents:
        return sent.text.strip()

def gen_what_response(user_input, wikipedia_summary):
    # Return the first sentence of the wikipedia summary
    doc = nlp(wikipedia_summary)
    
    # Return the first sentence
    for sent in doc.sents:
        return sent.text.strip()

def gen_when_response(user_input, wikipedia_summary):
    born_matches = re.findall(r'\bborn\b', user_input)
    born_question = False
    if len(born_matches) != 0:
        born_question = True
    
    if born_question:
        # Get the entity in the question
        named_entity = entity_search(user_input)
        
        sentences = sent_tokenize(wikipedia_summary)
        sentence_with_born = ''
        counted = False
        
        for sentence in sentences:
            if 'born' in sentence.lower() and counted == False:
                sentence_with_born = sentence
                counted = True

        # Grab date from sentence that contains the word born
        date = get_first_date(sentence_with_born)

        # If not in sentence, search entire text
        if date == None:
            date = get_first_date(wikipedia_summary)

        if date != None:
            return "{} was born on {}.".format(named_entity, date)
        else:
            return "I'm sorry, I don't know the answer."
    else:
        # Return the first sentence with a date in it
        sent_with_date = find_sentence_with_date(wikipedia_summary)
        if sent_with_date != None:
            return sent_with_date
        else:
            return "I'm sorry, I don't know the answer."

def gen_where_response(user_input, wikipedia_summary):
    location_patterns = r"\b((is|was|did|located)* (near|around|at|spanning)+)\b"
    response = find_sentence_with_regex(wikipedia_summary, location_patterns)
    if response != None:
        return response
    else:
        return "I'm sorry, I don't know the answer."

In [36]:
def generate_response(user_input, wikipedia_summary):
    question_patterns = {
        'what': r'\bwhat\b',
        'who': r'\bwho\b',
        'where': r'\bwhere\b',
        'when': r'\bwhen\b',
    }

    # Make user input lowercase
    user_input_lower = user_input.lower()

    # Identify question type
    question_type = None
    for qtype, pattern in question_patterns.items():
        if re.search(pattern, user_input_lower):
            question_type = qtype
            break

    response = "I'm sorry, I don't know the answer."
    # Now we have question type. Let's handle each case seperately
    if question_type == 'what':
        response = gen_what_response(user_input, wikipedia_summary)
    if question_type == 'who':
        response = gen_who_response(user_input, wikipedia_summary)
    if question_type == 'where':
        response = gen_where_response(user_input, wikipedia_summary)
    if question_type == 'when':
        response = gen_when_response(user_input, wikipedia_summary)

    return response

In [70]:
def respond_to_input(user_input):
    # First, check if the user said 'quit'
    continue_conv = end_condition(user_input)
    
    # Next, generate a wikipedia query based on the user input
    search_term = gen_search_terms(user_input)
    print("-----\nWIKIPEDIA QUERY: {}\n-----".format(search_term))

    # Next, get a wikipedia summary based on this query
    wikipedia_summary = fetch_wikipedia_summary(search_term)
    print("-----\nWIKIPEDIA SUMMARY:")
    print(wikipedia_summary)
    print("\n-----")

    # Last, generate a response based on user input and the wikipedia summary
    response = generate_response(user_input, wikipedia_summary)
    
    return response, continue_conv

In [71]:
def main():
    conversating = True

    # Start of conversation
    print("Please ask a question. Type 'exit' to exit.")

    # Speech loop
    while conversating:
        # Grab user input
        user_input = get_user_input()
        
        # Generate response, decide whether to continue the conversation
        response, continue_conv = respond_to_input(user_input)

        # If we're not continuing our conversation, break out of the loop.
        if not continue_conv:
            break

        # Print our response
        print("-----\nRESPONSE:\n{}\n-----".format(response))

    print("Thank you! Goodbye.")
main()

Please ask a question. Type 'exit' to exit.


>  Who is Jimmy Carter?


-----
WIKIPEDIA QUERY: Jimmy Carter
-----
-----
WIKIPEDIA SUMMARY:
James Earl Carter Jr. (born October 1, 1924) is an American politician and humanitarian who served as the 39th president of the United States from 1977 to 1981. A member of the Democratic Party, Carter was the 76th governor of Georgia from 1971 to 1975, and a Georgia state senator from 1963 to 1967. At age 99, he is both the oldest living former U.S. president and the longest-lived president in U.S. history. 
Carter was born and raised in Plains, Georgia. He graduated from the U.S. Naval Academy in 1946 and joined the U.S. Navy's submarine service. Carter returned home after his military service and revived his family's peanut-growing business. Opposing racial segregation, Carter supported the growing civil rights movement, and became an activist within the Democratic Party. He served in the Georgia State Senate from 1963 to 1967 and then as governor of Georgia from 1971 to 1975. As a dark-horse candidate not well known

>  When was Jimmy Carter born?


-----
WIKIPEDIA QUERY: Jimmy Carter
-----
-----
WIKIPEDIA SUMMARY:
James Earl Carter Jr. (born October 1, 1924) is an American politician and humanitarian who served as the 39th president of the United States from 1977 to 1981. A member of the Democratic Party, Carter was the 76th governor of Georgia from 1971 to 1975, and a Georgia state senator from 1963 to 1967. At age 99, he is both the oldest living former U.S. president and the longest-lived president in U.S. history. 
Carter was born and raised in Plains, Georgia. He graduated from the U.S. Naval Academy in 1946 and joined the U.S. Navy's submarine service. Carter returned home after his military service and revived his family's peanut-growing business. Opposing racial segregation, Carter supported the growing civil rights movement, and became an activist within the Democratic Party. He served in the Georgia State Senate from 1963 to 1967 and then as governor of Georgia from 1971 to 1975. As a dark-horse candidate not well known

>  Where were the twin towers located?


-----
WIKIPEDIA QUERY: twin towers
-----
-----
WIKIPEDIA SUMMARY:
The original World Trade Center (WTC) was a large complex of seven buildings in the Financial District of Lower Manhattan in New York City. It opened on April 4, 1973, and was destroyed during the September 11 attacks in 2001. At the time of their completion, the Twin Towers, including the original 1 World Trade Center (the North Tower) at 1,368 feet (417 m), and 2 World Trade Center (the South Tower) at 1,362 feet (415.1 m), were the tallest buildings in the world. Other buildings in the complex included the Marriott World Trade Center (3 WTC), 4 WTC, 5 WTC, 6 WTC, and 7 WTC. The complex contained 13,400,000 square feet (1,240,000 m2) of office space and, prior to its completion, was projected to accommodate an estimated 130,000 people.
The core complex was built between 1966 and 1975, at a cost of ~$400 million (equivalent to ~$3.80 billion in 2023). The idea was suggested by David Rockefeller to help stimulate urban r

>  exit


-----
WIKIPEDIA QUERY: exit
-----
-----
WIKIPEDIA SUMMARY:
Exit(s) may refer to:

-----
Thank you! Goodbye.
